In [45]:
%load_ext autoreload
%autoreload 2
%pylab inline
import pandas as pd
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Populating the interactive namespace from numpy and matplotlib


[autoreload of honk_sv.manage_audio failed: Traceback (most recent call last):
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 246, in check
    superreload(m, reload, self.old_objects)
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 385, in superreload
    update_generic(old_obj, new_obj)
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 324, in update_generic
    update(a, b)
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 279, in update_class
    if old_obj == new_obj:
ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
]
/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['mod']


In [46]:
import sys
sys.path.append("/home/muncok/DL/projects/sv_system/")

import pprint
pp = pprint.PrettyPrinter(indent=4)

from honk_sv import train as train
from honk_sv import model as mod
from honk_sv import dataset as dset
from honk_sv import system as svs
from honk_sv import dataloader as dloader

# Dataframes
command: disjoint speaker split

In [17]:
dataframe_dir = '/home/muncok/DL/dataset/SV_sets/dataframes/'
data_dir = '/home/muncok/DL/dataset/SV_sets/reddots_r2015q4_v1/wav/'
data_df = pd.read_pickle('/home/muncok/DL/dataset/SV_sets/dataframes/Reddots_Dataframe.pkl')

In [18]:
import honk_sv.train as hk
from honk_sv import model as mod
from honk_sv import dataset as dset
from honk_sv import system as svs
from honk_sv import dataloader as dloader

### Dataset split

In [19]:
all_spks = data_df.spk.unique()
all_sents = data_df.sent.unique()
uttrs_counts = data_df.spk.value_counts()

In [9]:
si_spks = list(uttrs_counts[uttrs_counts > 100].index)
sv_spks = list(uttrs_counts[uttrs_counts <= 100].index)

In [10]:
import random 

si_df = data_df[data_df.spk.isin(si_spks)]

# random sampling
si_random_train = si_df.sample(frac=0.8)
si_random_test = si_df.drop(index=si_random_train.index)
si_random_val = si_random_test
print("[random] train:{}, val:{}, test:{}".format(len(si_random_train), len(si_random_val), len(si_random_test)))

[random] train:11747, val:2937, test:2937


In [11]:
import csv
import random

tags = ['train', 'val', 'test']
# choose a si split
sets = {'train':si_random_train, 'val':si_random_val, 'test':si_random_test}
manifest_dir = "../manifests/reddots/"

for tag in tags:
    samples = []
    save_path = os.path.join(manifest_dir,'si_{}_{}_manifest.csv'.format("reddots", tag))
    with open(save_path, 'w') as f:
        for index, row in sets[tag].iterrows():
            file_path = os.path.join(data_dir, row.spk, row.file)
            label = si_spks.index(row.spk)
            sample = ','.join([file_path, str(label)])
            samples.append(sample)
        random.shuffle(samples)
        writer = csv.writer(f, delimiter='\n', quoting=csv.QUOTE_NONE)
        writer.writerow(samples)
        print("{} was written".format(save_path))

../manifests/reddots/si_reddots_train_manifest.csv was written
../manifests/reddots/si_reddots_val_manifest.csv was written
../manifests/reddots/si_reddots_test_manifest.csv was written


In [12]:
# sv write_manifest
samples = []
save_path = os.path.join(manifest_dir,'sv_{}_manifest.csv'.format("reddots"))
sv_df = data_df[data_df.spk.isin(sv_spks)]
with open(save_path, 'w') as f:
    for index, row in sv_df.iterrows():
        file_path = os.path.join(data_dir, row.spk, row.file)
        label = sv_spks.index(row.spk)
        sample = ','.join([file_path, str(label)])
        samples.append(sample)
    writer = csv.writer(f, delimiter='\n', quoting=csv.QUOTE_NONE)
    writer.writerow(samples)
    print("{} was written".format(save_path))

../manifests/reddots/sv_reddots_manifest.csv was written


# SI Train

In [77]:
model = "SimpleCNN"
dataset = "reddots"

global_config = dict(model=model, dataset=dataset,
                     no_cuda=False,  gpu_no=0,
                     n_epochs=100, batch_size=64,
                     lr=[0.01], schedule=[np.inf], dev_every=1, seed=0, use_nesterov=False,
                     cache_size=32768, momentum=0.9, weight_decay=0.00001,
                     num_workers=16, print_step=100,
                     splice_length=20
                     )

builder = train.ConfigBuilder(
                dset.SpeechDataset.default_config(),
                global_config)
parser = builder.build_argparse()
si_config = builder.config_from_argparse(parser)
si_config['model_class'] = mod.SimpleCNN
train.set_seed(si_config)

In [78]:
si_config['input_length'] = int(16000*0.2)
si_config['input_format'] = 'fft'
manifest_dir = "../manifests/reddots/"
for tag in ['train', 'val', 'test']:
    si_config['{}_manifest'.format(tag)]=os.path.join(manifest_dir,'si_{}_{}_manifest.csv'.format("reddots", tag))

In [79]:
from torch.autograd import Variable
import torch
import torch.nn as nn

si_config['n_labels'] = len(si_spks)
si_model = si_config['model_class']()
time_dim = si_config['input_length']//160+1
test_in = Variable(torch.zeros(1,1,time_dim,241), volatile=True)
test_out = si_model(test_in)
si_model.feat_size = test_out.size(1)
si_model.output = nn.Linear(test_out.size(1), si_config["n_labels"])

In [80]:
# si_model.load("models/reddots/si_reddots_frames_res8.pt")
si_config['n_epochs'] = 100
si_config['output_file'] = "../models/reddots/si_reddots_0.2s.pt"
hk.train(si_config, model=si_model)

train step #99 accuracy: 0.234375, loss: 3.061197519302368
epoch #0, final dev accuracy: 0.21521619946605644
saving best model...
train step #199 accuracy: 0.1875, loss: 2.975829839706421
train step #299 accuracy: 0.3125, loss: 2.740049123764038
epoch #1, final dev accuracy: 0.033961432112890926
train step #399 accuracy: 0.046875, loss: 3.865417242050171
train step #499 accuracy: 0.046875, loss: 3.7768826484680176
epoch #2, final dev accuracy: 0.05578995041952708
train step #599 accuracy: 0.0, loss: 3.880993366241455
train step #699 accuracy: 0.046875, loss: 3.722864866256714
epoch #3, final dev accuracy: 0.055831664759725404
train step #799 accuracy: 0.09375, loss: 3.684377670288086
train step #899 accuracy: 0.078125, loss: 3.6849100589752197
epoch #4, final dev accuracy: 0.057148646071701
train step #999 accuracy: 0.046875, loss: 3.6804306507110596


Process Process-9324:
Process Process-9323:
Process Process-9328:
Process Process-9327:
Process Process-9313:
Process Process-9318:
Process Process-9314:
Process Process-9316:
Process Process-9315:
Process Process-9325:
Process Process-9317:
Process Process-9319:
Process Process-9321:
Process Process-9320:
Process Process-9326:
Process Process-9322:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/muncok/anaconda3/envs/p

KeyboardInterrupt: 

  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 55, in _worker_loop
    samples = collate

  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 55, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/muncok/DL/projects/sv_system/honk_sv/dataset.py", line 312, in __getitem__
    return self.preprocess(self.audio_files[index]), self.audio_labels[index]
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/queues.py", line 341, in get
    with self._rlock:
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 55, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/multiprocessing/queues.py", line 341, in get
    with self._rlock:
  File "/home/muncok/anaconda3/envs/pytorch/lib/pyth

  File "/home/muncok/DL/projects/sv_system/honk_sv/manage_audio.py", line 36, in preprocess_audio
    data = fft_audio(data, 0.025, 0.010)
  File "/home/muncok/DL/projects/sv_system/honk_sv/manage_audio.py", line 48, in fft_audio
    win_length=win_length, window=windows['hamming'])
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/librosa/core/spectrum.py", line 152, in stft
    fft_window = util.pad_center(fft_window, n_fft)
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/librosa/util/utils.py", line 289, in pad_center
    return np.pad(data, lengths, **kwargs)
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/numpy/lib/arraypad.py", line 1377, in pad
    newmat = _append_const(newmat, pad_after, after_val, axis)
  File "/home/muncok/anaconda3/envs/pytorch/lib/python3.6/site-packages/numpy/lib/arraypad.py", line 139, in _append_const
    axis=axis)
KeyboardInterrupt
  File "/home/muncok/DL/projects/sv_system/honk_sv/

In [38]:
si_config['train_manifest']

'../manifests/reddots/si_reddots_train_manifest.csv'